In [1]:
import cv2
import numpy as np
#for distance calculation from the center of the hand to the finger calculation
from sklearn.metrics import pairwise 

In [2]:
#Now here we are going to create the avg background for the region of interest (ROI)
background = None
accumulated_weight = 0.5

#Region of Interest in a particular the capturing window
roi_top = 40 
roi_bottom = 320
roi_right = 350
roi_left = 650

In [3]:
#creating a function to find the avg background value
def calc_accum_avg(frame, accumulated_weight):
    
    global background #taking the global variable    
    
    if background is None:
        #taking the frame as background as copy and set the background as float
        background = frame.copy().astype('float') 
    return None

    #below function accumulates the weight according to the current frame
    #here we are updating the background using the accum_weight
    cv2.accumulateWeighted(frame, background, accumulated_weight)  


In [4]:
#Now will use thresholding to grab the hand segment from the ROI
#change the threshold value to low if the function gets more noise in the frames of background
def segment(frame, threshold_min=30):
    
    #calculating the absolute difference between the background and the current frame
    diff = cv2.absdiff(background.astype('uint8'), frame)

    #applying threshold to the passed frame image
    #for showing the white hand and black background
    ret,thresholded_img = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    #grabbing the external contours from the thresholded image
    contours, hierarchy = cv2.findContours(thresholded_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #if we grab not a single type of contour from the thresholded image
    if len(contours) == 0:
        return None
        
    #if we are able to get the contour, which would assumingly be the largest contour in ROI
    #i.e "OUR HAND" in the ROI
    else:
        hand_segment = max(contours, key = cv2.contourArea) #"key" is for choosing(treating) the largest contour viz our Hand
        return (thresholded_img, hand_segment)

In [5]:
########## NOW HERE WE WILL FIND THE FINGERS HELD UP IN THE REGION OF INTEREST FROM OUR HAND #############

#HERE WE WILL UTILIZE THE CONVEX HULL FOR IDENTIFING THE FINGERS 
#AND HERE THE SET OF POINTS IS OUR "thresholded_img" and the "hand_segment"(external contour information)

def count_fingers(thresholded_img, hand_segment):
    
    #creating the convexHull
    conv_hull = cv2.convexHull(hand_segment)
    
    #extreme Points
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0]) #most extreme top point of the convex hull
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0]) #most extreme bottom point of the convex hull
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0]) #most extreme left point of the convex hull
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0]) #most extreme right point of the convex hull
    
    #finding center of the hand by calculation
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    #finding the euclidean distances for all the extreme points from the center of our hand 
    distance = pairwise.euclidean_distances([(cX,cY)], Y = [left, right, top, bottom])[0]
    
    #getting the max distance of all the extreme points from the center of the hand
    max_distance = distance.max() 
    
    #create the circle with the 70% of the max_distance from the center of our hand
    radius = int(0.7 * max_distance) 
    circumference = (2*np.pi*radius)
    
    #creating ROI for that circle
    circular_roi = np.zeros(thresholded_img.shape[:2], dtype = 'uint8')
    #draw the circular region of interest
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10) 
    
    #using bitwise with the circle ROI as the mask
    circular_roi = cv2.bitwise_and(thresholded_img, thresholded_img, mask=circular_roi)
    
    #grab all the contours which are outside of the circular ROI which has a center as the center of out hand
    contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    #here we will count the contours which are outside that circular ROI 
    count = 0 
    for cnt in contours:
    
        (x,y,w,h) = cv2.boundingRect(cnt)
        #MAKING THE LIMITATIONS IF THE OUTSIDE ROI IS NOT WRIST BUT FINGERS OF OUR HAND
        out_of_wrist = (cY+(cY*0.25))>(y+h)
    
        #AND ALSO MAKING SURE THAT THERE NO OTHER NOISE WHICH IS WAY OUTSIDE OF THE CIRCUMFERENCE OF THE CIRCLE  
        #should not go the points outside the 25% of the circumference
        limit_points = ((circumference*0.25)> cnt.shape[0])
    
        if out_of_wrist and limit_points:
            count+= 1
      #THEN WE WILL RETURN THE REST OF THE CONTOURS LEFT AND USE IT AS THE "COUNT" OF THE FINGERS OF OUR HAND
    return count


In [7]:
#Bringing all together by adding the camera capture, etc

#WE WILL HAVE "TWO WINDOWS" WHICH ARE

# (1.) SMALLER WINDOW
#*THIS WILL BE THE THRESHOLDED WINDOW WHICH WE WOULD OUTPUT THE THRESHOLDED HAND LIVE VIDEO CAPTURE FROM THE ROI

# (2.) BIGGER WINDOW
#*THIS WILL BE SHOWING THE ROI FOR ENTERING THE HAND IN IT AND WILL ALSO DISPLAY THE NO. OF FINGERS OF THAT ENTERING HAND


cam = cv2.VideoCapture(0)
num_frames = 0

while True:

    ret, frame = cam.read()
    frame_copy = frame.copy()

    #getting the overall ROI 
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    #converting the frame to grayscale for better identification
    grayscale_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    #bluring to get a good avg background
    grayscale_frame = cv2.GaussianBlur(grayscale_frame, (9, 9), 0) #9x9 kernel

    #IF THE NUMBER OF FRAMES IS 60 WE WILL ACCUMULATE THE WEIGHT AND FIND THE AVG ACCUMULATED WEIGHT    
    if num_frames < 60:
        calc_accum_avg(grayscale_frame, accumulated_weight)#(defaulted value) accumulated_weight = 0.5
    
        #AND IF THE NUMBER OF FRAMES IS NOT EQUAL TO 60 FRAMES THAN THE FOLLOWING WILL BE PRINTED IN THE FRAME UNTIL
        #THERE ARE TOTAL 60 FRAMES  
        if num_frames <= 59:
            cv2.putText(frame_copy, 'WAIT...! GETTING BACKGROUND', (300, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
            #cv2.imshow('Finger Counter', frame_copy) #THIS WILL DISPLAY THE FRAME WITH TEXT WRITTEN IN ABOVE CODE
    
    #WHEN THE NUMBER OF FRAMES IS 60 AND THE AVERAGE ACCUMULATED WEIGHT IS FOUND 
    #THEN WE COULD START ENTERING OUR HAND INTO THE FRAME'S REGION OF INTEREST(ROI)  
    else:
        
        hand = segment(grayscale_frame)#this function segment gives us the thresholded_img and hand_segment
        
        if hand is not None:
            
            #tuple unpacking the "hand"
            thresholded_img, hand_segment = hand
        
            #DRAWS CONTOURS AROUND REAL HAND IN THE LIVE STREAM VIDEO CAPTURE
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 255, 255), 3)
            
            #COUNTING FINGERS
            fingers = count_fingers(thresholded_img, hand_segment)
            
            #SHOWING THE TEXT IN THE FRAME
            cv2.putText(frame_copy, str(fingers), (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
            
            #Displaying the Thresholded video capture
            cv2.imshow("Thresholded_ROI", thresholded_img)
            
    #DRAWING THE RECTANGLE FOR THE REGION OF INTEREST IN THE BIGGER WINDOW
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0, 0, 255), 5)
    
    num_frames += 1   #INCREMENTING THE NUMBER OF FRAMES FOR GETTING TOTAL 60 FRAMES FOR FINDING THE ACCUMULATED WEIGHT  
    
    cv2.imshow('Finger_Counter', frame_copy)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()   